In [55]:
import requests
import time
import datetime
import dateutil.parser
import csv
#import setting　#別で管理していない人は、KEY=直書きしてね。
import json

#KEY  = setting.TWITTER_BEARER_TOKEN ## DeveloperPortalで取得したBearer Token。お試し時は直書きでもOK。
bearer_token = "×××××××××××××××××××××××××××××××××××" # 自分のAPIキーを入力
headers = {"Authorization": "Bearer {}".format(bearer_token)}
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST') #JST設定
url  = 'https://api.twitter.com/2/tweets/search/all'  #/search/recentをsearch/allにしたら過去のツイートが取れる

def getTweetByText(params,path):
    
    params = params
    response = requests.request('GET',
                           url,
                           headers= headers,
                           params = params
                          )

    result = response.json()
    count = 0
    file_tweet = True
    
    while True:
        print("-----------------------------------------------------------------------")
        if (response.status_code == 200):
            print("ステータスコードは" + str(response.status_code))
            if ('data' in result):
                tweet_data = []
                for tweet in result['data']:
                    if ('referenced_tweets' not in tweet):
                        add_tweet = {'referenced_tweets':[{'type': 'original', 'id': tweet['id']}]}
                        tweet.update(add_tweet)
                    tweet_data.append([tweet["id"],
                                       tweet["text"],
                                       tweet['author_id'],
                                       dateutil.parser.parse(tweet["created_at"]).astimezone(JST).strftime('%Y-%m-%d %H:%M:%S'),
                                       tweet["public_metrics"]["retweet_count"],
                                       tweet["public_metrics"]["reply_count"],
                                       tweet["public_metrics"]["like_count"],
                                       tweet["public_metrics"]["quote_count"], 
                                       [d.get('type') for d in tweet['referenced_tweets']][0],
                                       [d.get('id') for d in tweet['referenced_tweets']][0]
                                      ])
                    count += 1
                    
                with open(path, 'a',newline='',encoding='utf-8') as f:
                    writer = csv.writer(f)
                    if(file_tweet):
                        writer.writerow(['id',
                                         'text',
                                         'author_id',
                                         'created_at',
                                         "retweet_count",
                                         "reply_count",
                                         "like_count",
                                         "quote_count",
                                         'referenced_type',
                                         'referenced_id'])
                    writer.writerows(tweet_data)
                file_tweet = False

        
            if ('next_token' in result['meta']):
                params['next_token'] = result['meta']['next_token']
                print('検索結果は' + str(result['meta']['result_count']) + '件でした')
                
            if ('next_token' not in result['meta']):
                print("ツイート数：" + str(count))
                break;
                    
        else:
            print("ステータスコードは" + str(response.status_code))
            time.sleep(2)
        
                        
        print("ツイート数：" + str(count))
        
        if('x-rate-limit-remaining' in response.headers):
            print("残りアクセス可能回数：" + response.headers['x-rate-limit-remaining'])   #残りアクセス回数（最大300回）
            dt_now = datetime.datetime.now()               #現在時間取得
            dt_reset = datetime.datetime.fromtimestamp(int(response.headers['x-rate-limit-reset'])) #エポック秒数を日付に変換
            print(dt_now)
            print(dt_reset)
            remaining_time = dt_reset - dt_now
            print("アクセス回数制限解除まで" + str(remaining_time.seconds) + "秒")    #アクセス回数制限解除までの秒数

            if(int(response.headers['x-rate-limit-remaining']) == 0):
                time.sleep(remaining_time.seconds+2)
        
        response = requests.request('GET',
                        url,
                        headers= headers,
                        params = params
                      )
        
        result = response.json()

In [62]:
params = {'query'        : '××', # 検索したいワードを入力
          'tweet.fields' : 'author_id,public_metrics,created_at', #created_at　←　ISO8601：日付と時刻の表記に関するISOの国際規格．日付と時刻をT記号で区切り，時分秒の表記はコロン (:) で区切る．末尾にUTCであればZ記号，他のタイムゾーンならばプラス記号 (+) あるいはマイナス記号 (-) で区切って続ける．
          'max_results'  : 500,                         #1回のリクエストで最大500件までツイートを取得することができる.    ←デフォルト10件
          'expansions'   : 'author_id,referenced_tweets.id',
          'start_time'   : '2017-01-01T00:00:00.00Z',  #検索期間開始点　　　←　この2つ書かない場合
          'end_time'     : '2022-09-30T00:00:00.00Z'    #検索期間終了点　　　←　デフォルト30日前
          }
path = "××" #csv保存先のパスを入れる

getTweetByText(params,path)

-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：493
残りアクセス可能回数：290
2022-10-10 00:07:15.139745
2022-10-10 00:13:08
アクセス回数制限解除まで352秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は494件でした
ツイート数：987
残りアクセス可能回数：291
2022-10-10 00:07:16.841628
2022-10-10 00:13:10
アクセス回数制限解除まで353秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は491件でした
ツイート数：1478
残りアクセス可能回数：289
2022-10-10 00:07:18.565688
2022-10-10 00:13:08
アクセス回数制限解除まで349秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は485件でした
ツイート数：1963
残りアクセス可能回数：288
2022-10-10 00:07:20.193575
2022-10-10 00:13:08
アクセス回数制限解除まで347秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は485件でした
ツイート数：2448
残りアクセス可能回数：287
2022-10-10 00:07:21.940231
2022-10-10 00:13:08
アクセス回数制限解除まで346秒
--------------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は477件でした
ツイート数：22092
残りアクセス可能回数：268
2022-10-10 00:08:41.038325
2022-10-10 00:13:08
アクセス回数制限解除まで266秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は471件でした
ツイート数：22563
残りアクセス可能回数：269
2022-10-10 00:08:42.900323
2022-10-10 00:13:10
アクセス回数制限解除まで267秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は483件でした
ツイート数：23046
残りアクセス可能回数：267
2022-10-10 00:08:44.527787
2022-10-10 00:13:08
アクセス回数制限解除まで263秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は490件でした
ツイート数：23536
残りアクセス可能回数：268
2022-10-10 00:08:46.366000
2022-10-10 00:13:10
アクセス回数制限解除まで263秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は485件でした
ツイート数：24021
残りアクセス可能回数：266
2022-10-10 00:08:48.009684
2022-10-10 00:13:08
アクセス回数制限解除まで259秒
-------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は478件でした
ツイート数：43155
残りアクセス可能回数：243
2022-10-10 00:09:56.722081
2022-10-10 00:13:08
アクセス回数制限解除まで191秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は495件でした
ツイート数：43650
残りアクセス可能回数：250
2022-10-10 00:09:58.577623
2022-10-10 00:13:10
アクセス回数制限解除まで191秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は491件でした
ツイート数：44141
残りアクセス可能回数：242
2022-10-10 00:10:00.432055
2022-10-10 00:13:08
アクセス回数制限解除まで187秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は498件でした
ツイート数：44639
残りアクセス可能回数：249
2022-10-10 00:10:02.832783
2022-10-10 00:13:10
アクセス回数制限解除まで187秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は487件でした
ツイート数：45126
残りアクセス可能回数：241
2022-10-10 00:10:04.588328
2022-10-10 00:13:08
アクセス回数制限解除まで183秒
-------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は489件でした
ツイート数：64836
残りアクセス可能回数：223
2022-10-10 00:11:18.033268
2022-10-10 00:13:08
アクセス回数制限解除まで109秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は498件でした
ツイート数：65334
残りアクセス可能回数：226
2022-10-10 00:11:20.071994
2022-10-10 00:13:10
アクセス回数制限解除まで109秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は496件でした
ツイート数：65830
残りアクセス可能回数：222
2022-10-10 00:11:21.887936
2022-10-10 00:13:08
アクセス回数制限解除まで106秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は490件でした
ツイート数：66320
残りアクセス可能回数：221
2022-10-10 00:11:23.974953
2022-10-10 00:13:08
アクセス回数制限解除まで104秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は494件でした
ツイート数：66814
残りアクセス可能回数：220
2022-10-10 00:11:25.740923
2022-10-10 00:13:08
アクセス回数制限解除まで102秒
-------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は491件でした
ツイート数：86608
残りアクセス可能回数：207
2022-10-10 00:12:42.027057
2022-10-10 00:13:08
アクセス回数制限解除まで25秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は497件でした
ツイート数：87105
残りアクセス可能回数：198
2022-10-10 00:12:44.132182
2022-10-10 00:13:10
アクセス回数制限解除まで25秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：87598
残りアクセス可能回数：197
2022-10-10 00:12:45.836705
2022-10-10 00:13:10
アクセス回数制限解除まで24秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：88091
残りアクセス可能回数：206
2022-10-10 00:12:47.425024
2022-10-10 00:13:08
アクセス回数制限解除まで20秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は488件でした
ツイート数：88579
残りアクセス可能回数：205
2022-10-10 00:12:49.087012
2022-10-10 00:13:08
アクセス回数制限解除まで18秒
------------------------------------------------------------

...

-----------------------------------------------------------------------
ステータスコードは200
検索結果は490件でした
ツイート数：559748
残りアクセス可能回数：81
2022-10-10 00:41:42.494233
2022-10-10 00:43:13
アクセス回数制限解除まで90秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は478件でした
ツイート数：560226
残りアクセス可能回数：63
2022-10-10 00:41:44.007191
2022-10-10 00:43:15
アクセス回数制限解除まで90秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は497件でした
ツイート数：560723
残りアクセス可能回数：80
2022-10-10 00:41:45.785323
2022-10-10 00:43:13
アクセス回数制限解除まで87秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は489件でした
ツイート数：561212
残りアクセス可能回数：79
2022-10-10 00:41:47.527985
2022-10-10 00:43:13
アクセス回数制限解除まで85秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は497件でした
ツイート数：561709
残りアクセス可能回数：62
2022-10-10 00:41:49.360331
2022-10-10 00:43:15
アクセス回数制限解除まで85秒
------------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は494件でした
ツイート数：581416
残りアクセス可能回数：41
2022-10-10 00:42:58.831801
2022-10-10 00:43:15
アクセス回数制限解除まで16秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は498件でした
ツイート数：581914
残りアクセス可能回数：40
2022-10-10 00:43:00.334404
2022-10-10 00:43:15
アクセス回数制限解除まで14秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：582407
残りアクセス可能回数：59
2022-10-10 00:43:02.092947
2022-10-10 00:43:13
アクセス回数制限解除まで10秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は485件でした
ツイート数：582892
残りアクセス可能回数：39
2022-10-10 00:43:03.844665
2022-10-10 00:43:15
アクセス回数制限解除まで11秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：583385
残りアクセス可能回数：58
2022-10-10 00:43:05.589676
2022-10-10 00:43:13
アクセス回数制限解除まで7秒
-------------------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は499件でした
ツイート数：603157
残りアクセス可能回数：284
2022-10-10 00:44:13.750225
2022-10-10 00:58:15
アクセス回数制限解除まで841秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は496件でした
ツイート数：603653
残りアクセス可能回数：283
2022-10-10 00:44:15.359743
2022-10-10 00:58:15
アクセス回数制限解除まで839秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は498件でした
ツイート数：604151
残りアクセス可能回数：280
2022-10-10 00:44:17.096630
2022-10-10 00:58:16
アクセス回数制限解除まで838秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は489件でした
ツイート数：604640
残りアクセス可能回数：282
2022-10-10 00:44:18.650827
2022-10-10 00:58:15
アクセス回数制限解除まで836秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は489件でした
ツイート数：605129
残りアクセス可能回数：281
2022-10-10 00:44:20.088770
2022-10-10 00:58:15
アクセス回数制限解除まで834秒
--------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は492件でした
ツイート数：624958
残りアクセス可能回数：261
2022-10-10 00:45:25.532724
2022-10-10 00:58:16
アクセス回数制限解除まで770秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は489件でした
ツイート数：625447
残りアクセス可能回数：259
2022-10-10 00:45:27.007607
2022-10-10 00:58:15
アクセス回数制限解除まで767秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は481件でした
ツイート数：625928
残りアクセス可能回数：258
2022-10-10 00:45:28.545645
2022-10-10 00:58:15
アクセス回数制限解除まで766秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は498件でした
ツイート数：626426
残りアクセス可能回数：257
2022-10-10 00:45:29.993382
2022-10-10 00:58:15
アクセス回数制限解除まで765秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は495件でした
ツイート数：626921
残りアクセス可能回数：256
2022-10-10 00:45:31.448042
2022-10-10 00:58:15
アクセス回数制限解除まで763秒
--------------------------------------------------

-----------------------------------------------------------------------
ステータスコードは200
検索結果は493件でした
ツイート数：646660
残りアクセス可能回数：234
2022-10-10 00:46:37.211795
2022-10-10 00:58:15
アクセス回数制限解除まで697秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は499件でした
ツイート数：647159
残りアクセス可能回数：242
2022-10-10 00:46:39.106392
2022-10-10 00:58:16
アクセス回数制限解除まで696秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は494件でした
ツイート数：647653
残りアクセス可能回数：233
2022-10-10 00:46:40.574293
2022-10-10 00:58:15
アクセス回数制限解除まで694秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は494件でした
ツイート数：648147
残りアクセス可能回数：241
2022-10-10 00:46:42.219435
2022-10-10 00:58:16
アクセス回数制限解除まで693秒
-----------------------------------------------------------------------
ステータスコードは200
検索結果は497件でした
ツイート数：648644
残りアクセス可能回数：232
2022-10-10 00:46:43.660459
2022-10-10 00:58:15
アクセス回数制限解除まで691秒
--------------------------------------------------